In [1]:
########################################################################################################################
# This script extracts the demographic information of patients.
########################################################################################################################

In [2]:
########################################################################################################################
# Import packages
########################################################################################################################
import numpy as np
import os
import pandas as pd
import warnings
from itertools import product
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

In [3]:
########################################################################################################################
# USER-SPECIFIC SETTING
# Cs: Different numbers of feature encounteres to be included
# Ds: Different maximum widths of the look-back window in days
########################################################################################################################
Cs : list[int] = [1, 2, 3, 4]
Ds : list[int] = [60, 120, 180]

In [ ]:
########################################################################################################################
# USER_SPECIFIC SETTING
# TARGET_DIR_PATH: Path of the directory storing the target datasets created in P01_Subject_Inclusion.ipynb
########################################################################################################################
TARGET_DIR_PATH: str = '../00_Data/02_Processed_Data/Targets/'
OUT_FILE_PATH: str = 'Demographics.csv'

In [ ]:
########################################################################################################################
# Create an empty list to store summarizied statistics
########################################################################################################################
stat_list: list[dict] = []

In [ ]:
########################################################################################################################
# Loop over the experiment configurations Cs and Ds
########################################################################################################################
for conf_idx, (C, D) in enumerate(product(Cs, Ds), 1):
    log_head: str = f'[{conf_idx}. C={C}; D={D}] '
    if C == 1 and Ds.index(D) > 0:      # When C=1, all D values are the same
        continue

    ####################################################################################################################
    # Load the target dataset
    ####################################################################################################################
    data_path: str = os.path.join(TARGET_DIR_PATH, f'{C}_encounters_{D}_days_v1.csv')
    assert os.path.exists(data_path)
    df: pd.DataFrame = pd.read_csv(data_path)
    cur_stat: dict = {'C': C, 'D': D}
    
    ####################################################################################################################
    # Extract the prevalence rates
    ####################################################################################################################
    N: int = df.shape[0]
    count_dict: dict[int, int] = df['OutcomeLabel'].value_counts().sort_index(ascending=False).to_dict()
    pct_dict: dict[int, float] = df['OutcomeLabel'].value_counts(normalize=True).sort_index(ascending=False).to_dict()
    pct_dict = {k: f'{round(v*100, 1)}%' for k, v in pct_dict.items()}
    print(f'{log_head}Outcome counts: {count_dict}')
    print(f'{log_head}Outcome percentages: {pct_dict}')
    cur_stat |= {'N': N, 
                 'Outcome_Y_n': count_dict[1], 
                 'Outcome_N_n': count_dict[0],
                 'Outcome_Y_pct': pct_dict[1],
                 'Outcome_N_pct': pct_dict[0]}
    print('-'*120)
    
    ####################################################################################################################
    # Specify the needed columns
    ####################################################################################################################
    needed_cols: list[str] = ['Sex^0=Female', 'Sex^1=Male', 'Sex^2=Other']                          # Sex
    needed_cols += sorted([col for col in df.columns if col.startswith('GenderIdentity^')])         # Gender
    needed_cols += sorted([col for col in df.columns if col.startswith('Race^')])                   # Race
    needed_cols += ['Ethnicity^1=Hispanic or Latino']                                               # Ethnicity
    needed_cols += ['OutcomeEncAge']
    assert set(needed_cols).issubset(df.columns)

    ####################################################################################################################
    # Get sex statistics
    ####################################################################################################################
    sex_f_n: int = df['Sex^0=Female'].sum().astype(int)
    sex_m_n: int = df['Sex^1=Male'].sum().astype(int)
    sex_o_n: int = (df['Sex^2=Other'].sum() + df['Sex^2=Other'].isna().sum()).astype(int)
    for n, n_name in zip([sex_f_n, sex_m_n, sex_o_n],
                         ['Sex_Female_n', 'Sex_Male_n', 'Sex_Other_n']):
        cur_stat |= {n_name: n} | {n_name.replace('_n', '_pct'): f'{round(n * 100 / N, 1)}%'}

    ####################################################################################################################
    # Get sex-specific outcome distribution
    ####################################################################################################################
    df_F: pd.DataFrame = df[df['Sex^0=Female'] == 1]
    count_F: int = df_F['OutcomeLabel'].value_counts().to_dict()[1]
    pct_F: float = df_F['OutcomeLabel'].value_counts(normalize=True).to_dict()[1] * 100
    print(f'{log_head}FEMALE; Outcome counts: {count_F} ({pct_F:.1f}%)')

    df_M: pd.DataFrame = df[df['Sex^1=Male'] == 1]
    count_M: int = df_M['OutcomeLabel'].value_counts().to_dict()[1]
    pct_M: float = df_M['OutcomeLabel'].value_counts(normalize=True).to_dict()[1] * 100
    print(f'{log_head}MALE; Outcome counts: {count_M} ({pct_M:.1f}%)')

    cur_stat |= {'Outcome_Sex_Female_n': count_F,
                 'Outcome_Sex_Female_pct': f'{pct_F:.1f}%',
                 'Outcome_Sex_Male_n': count_M,
                 'Outcome_Sex_Female_pct': f'{pct_M:.1f}%'}
    
    ####################################################################################################################
    # Get gender identity statistics
    ####################################################################################################################
    gen_f_n: int = df['GenderIdentity^0=Female'].sum().astype(int)
    gen_m_n: int = df['GenderIdentity^1=Male'].sum().astype(int)
    gen_tm_n: int = df['GenderIdentity^2=Transgender Male / Female-to-Male'].sum().astype(int)
    gen_tf_n: int = df['GenderIdentity^3=Transgender Female / Male-to-Female'].sum().astype(int)
    gen_o_n: int = df['GenderIdentity^0=Female'].isna().sum().astype(int)
    for n, n_name in zip([gen_f_n, gen_m_n, gen_tm_n, gen_tf_n, gen_o_n],
                         ['Gender_Female_n', 'Gender_Male_n', 'Gender_TransMale_n', 'Gender_TransFemale_n', 'Gender_Other_n']):
        cur_stat |= {n_name: n} | {n_name.replace('_n', '_pct'): f'{round(n * 100 / N, 1)}%'}

    ####################################################################################################################
    # Get race statistics
    ####################################################################################################################
    race_w_n: int = df['Race^5=White'].sum().astype(int)
    race_b_n: int = df['Race^2=Black or African American'].sum().astype(int)
    race_a_n: int = df['Race^1=Asian'].sum().astype(int)
    race_aian_n: int = df['Race^0=American Indian or Alaska Native'].sum().astype(int)
    race_nhopi_n: int = df['Race^3=Native Hawaiian or Other Pacific Islander'].sum().astype(int)
    race_mul_n: int = df['Race^6=Multiple Races'].sum().astype(int)
    race_o_n: int = (df['Race^4=Other Race'].sum() + df['Race^4=Other Race'].isna().sum()).astype(int)
    for n, n_name in zip([race_w_n, race_b_n, race_a_n, race_aian_n, race_nhopi_n, race_mul_n, race_o_n],
                         ['Race_White_n', 'Race_Black_n', 'Race_Asian_n', 'Race_AIAN_n', 'Race_NHOPI_n', 'Race_Multiple_n', 'Race_Other_n']):
        cur_stat |= {n_name: n} | {n_name.replace('_n', '_pct'): f'{round(n * 100 / N, 1)}%'}

    ####################################################################################################################
    # Get ethnicity statistics
    ####################################################################################################################
    eth_h_n: int = df['Ethnicity^1=Hispanic or Latino'].sum().astype(int)
    eth_nh_n: int = df[df['Ethnicity^1=Hispanic or Latino'] == 0].shape[0]
    eth_o_n: int = df['Ethnicity^1=Hispanic or Latino'].isna().sum().astype(int)
    for n, n_name in zip([eth_h_n, eth_nh_n, eth_o_n],
                         ['Ethnicity_Hispanic_n', 'Ethnicity_NonHispanic_n', 'Ethnicity_Other_n']):
        cur_stat |= {n_name: n} | {n_name.replace('_n', '_pct'): f'{round(n * 100 / N, 1)}%'}

    ####################################################################################################################
    # Get age statistics
    ####################################################################################################################
    age_min: float = df['OutcomeEncAge'].min()
    age_max: float = df['OutcomeEncAge'].max()
    age_mean: float = df['OutcomeEncAge'].mean()
    age_median: float = df['OutcomeEncAge'].median()
    for n, n_name in zip([age_min, age_max, age_mean, age_median],
                         ['Age_Min', 'Age_Max', 'Age_Mean', 'Age_Median']):
        cur_stat |= {n_name: n}

    ####################################################################################################################
    # Append cur_stat to stat_list
    ####################################################################################################################
    stat_list.append(cur_stat)

########################################################################################################################
# Save the concatenated result as a CSV file
########################################################################################################################
df_stat: pd.DataFrame = pd.DataFrame.from_records(stat_list)
df_stat.to_csv(OUT_FILE_PATH, index=False)
